<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/annotation_import/video.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>

<a href="https://github.com/Labelbox/labelbox-python/tree/develop/examples/annotation_import/video.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td> 

# Video Annotation Import

* Annotations must be created and uploaded using NDJSON
* Supported annotations that can be uploaded through the SDK:
    * Bounding box
    * Point
    * Polyline 
    * Radio classifications 
    * Checklist classifications 
* **NOT** supported:
    * Polygons 
    * Segmentation masks
    * Free form text classifications

Please note that this list of unsupported annotations only refers to limitations for importing annotations. For example, when using the Labelbox editor, segmentation masks can be created and edited on video assets.

### Setup

In [28]:
!pip install -q 'labelbox[data]'

In [29]:
import labelbox as lb
import uuid

### Replace with your API key 
Guides on [Create an API key](https://docs.labelbox.com/docs/create-an-api-key)

In [31]:
# Add your api key
API_KEY = None
client = lb.Client(api_key=API_KEY)

## Supported annotations for video
Only NDJSON annotations are supported with video assets

### Supported NDJSON annotations

In [32]:
######## Bounding box  ###########

# NDJSON
bbox_annotation_ndjson = {
    "name" : "bbox_video",
    "segments" : [{
        "keyframes" : [
            {
              "frame": 13,
              "bbox" : {
                "top": 146.0,
                "left": 98.0,
                "height": 382.0,
                "width": 341.0
              }  
           },
           {
              "frame": 14,
              "bbox" : {
                "top": 146.0,
                "left": 98.0,
                "height": 382.0,
                "width": 341.0
              }  
           },
           {
              "frame": 15,
              "bbox" : {
                "top": 146.0,
                "left": 98.0,
                "height": 382.0,
                "width": 341.0
              }  
           }
        ]
      }
    ]
}

In [33]:
######## Point ########

#NDJSON
point_annotation_ndjson = {
    "name": "point_video", 
    "segments": [{
        "keyframes": [{
            "frame": 17,
            "point" : {
                "x": 660.134 ,
                "y": 407.926
            }
        }]
    }] 
}

In [34]:
######## Polyline ########

# NDJSON (frame based annotations are supported with NDJSON format)
polyline_frame_annotation_ndjson = {
  "name": "line_video_frame", 
  "segments": [
      {
        "keyframes": [
          {
            "frame": 5,
            "line": [{
              "x": 680,
              "y": 100
            },{
              "x": 100,
              "y": 190
            },{
              "x": 190,
              "y": 220
            }]
          },
          {
            "frame": 12,
            "line": [{
              "x": 680,
              "y": 280
            },{
              "x": 300,
              "y": 380
            },{
              "x": 400,
              "y": 460
            }]
          },
          {
            "frame": 20,
            "line": [{
              "x": 680,
              "y": 180
            },{
              "x": 100,
              "y": 200
            },{
              "x": 200,
              "y": 260
            }]
          }
        ]
      },
      {
        "keyframes": [
          {
            "frame": 24,
            "line": [{
              "x": 300,
              "y": 310
            },{
              "x": 330,
              "y": 430
            }]
          },
          {
            "frame": 45,
            "line": [{
              "x": 600,
              "y": 810
            },{
              "x": 900,
              "y": 930
            }]
          }
        ]
      }
    ]
}

In [35]:
######## classifications ########

## NDJSON

## frame specific
frame_checklist_classification_ndjson = {
    "name": "checklist_class", 
    "answer": [
        { "name": "first_checklist_answer" , "frames": [{"start": 29, "end": 35 }, {"start": 48, "end": 65}]},
        { "name": "second_checklist_answer", "frames": [{"start": 29, "end": 35 }, {"start": 48, "end": 65}]} 
  ]      
}

# Global 
global_radio_classification_ndjson = {
    "name": "radio_class_global", 
    "answer": { "name": "first_radio_answer" }
}



In [36]:
########## Nested Global Classification ########### 

nested_classification = {
  'name': 'radio_question_nested',
  'answer': {'name': 'first_radio_question'},
  'classifications' : [
    {'name': 'sub_question_radio', 'answer': {'name': 'sub_answer'}}
   ]
}

In [37]:
########## Classifications under frame base tools ##########

# Frame base nested classifications do not support using the feature's name to extract ontology features. 
# For this single case we are going to use the classification's featureSchemaId and the answers' featureSchemaId 
# We will update the annotation object with the featureSchemaIds on step 5 after we create the ontology in step 2


frame_bbox_with_checklist_subclass_ndjson = {
    "name": "bbox_class",
    "segments": [{
        "keyframes": [
            {
            "frame": 10,
            "bbox": {
                "top": 146.0,
                "left": 98.0,
                "height": 382.0,
                "width": 341.0
              },
            "classifications" : [
              {'schemaId' : '', 'answer' : {'schemaId': '' }}
            ]     
          },
          {  
          "frame": 11,
            "bbox": {
                "top": 146.0,
                "left": 98.0,
                "height": 382.0,
                "width": 341.0
              },
            "classifications" : [
              {'schemaId' : '', 'answer' : {'schemaId': '' }}
            ]  
          },
          {  
          "frame": 13,
            "bbox": {
                "top": 146.0,
                "left": 98.0,
                "height": 382.0,
                "width": 341.0
              },
            "classifications" : [
              {'schemaId' : '', 'answer' : {'schemaId': '' }}
            ]  
          }
        ]
      }
    ]
}

## Upload Annotations - putting it all together

### Step 1: Import data rows into Catalog

In [38]:
asset = {
    "row_data": "https://storage.googleapis.com/labelbox-datasets/video-sample-data/sample-video-2.mp4", 
    "global_key": str(uuid.uuid4()),
    "media_type": "VIDEO"
}

dataset = client.create_dataset(name="video_demo_dataset")
data_row = dataset.create_data_row(asset)
print(data_row.uid)
print(data_row)

cldt0g5pd06gd07z22m2i6k66
<DataRow {
    "created_at": "2023-02-06 16:11:29+00:00",
    "external_id": null,
    "global_key": "cdd1739b-c91f-4e11-b81c-bccbe12be911",
    "media_attributes": {},
    "metadata": [],
    "metadata_fields": [],
    "row_data": "https://storage.googleapis.com/labelbox-datasets/video-sample-data/sample-video-2.mp4",
    "uid": "cldt0g5pd06gd07z22m2i6k66",
    "updated_at": "2023-02-06 16:11:29+00:00"
}>


### Step 2: Create/select an ontology
Your project should have the correct ontology setup with all the tools and classifications supported for your annotations, and the tool names and classification instructions should match the `name`/`instructions` fields in your annotations to ensure the correct feature schemas are matched.

For example, when we create the bounding box annotation [above](https://colab.research.google.com/drive/1S3HBa10jwfFFNB71Uid-crUDdhfEGnXh#scrollTo=suburban-crowd&line=10&uniqifier=1), we provided the `name` as `bbox_video`. Now, when we setup our ontology, we must ensure that the name of my bounding box tool is also `bbox_video`. The same alignment must hold true for the other tools and classifications we create in our ontology.


[Documentation for reference ](https://docs.labelbox.com/reference/import-text-annotations)

In [39]:
ontology_builder = lb.OntologyBuilder(
    tools=[
        lb.Tool(tool=lb.Tool.Type.BBOX, name="bbox_video"),
        lb.Tool(tool=lb.Tool.Type.POINT, name="point_video"),
        lb.Tool(tool=lb.Tool.Type.LINE, name="line_video_frame"),
        lb.Tool(
          tool=lb.Tool.Type.BBOX, name="bbox_class",
          classifications=[
            lb.Classification(
              class_type=lb.Classification.Type.RADIO,
              instructions="bbox_radio", 
              scope = lb.Classification.Scope.INDEX,
              options=[
                lb.Option(value="bbox_radio_answer_1"),
                lb.Option(value="bbox_radio_answer_2"),
                lb.Option(value="bbox_radio_answer_3")
              ]
            )
          ]
        )
    ],
    classifications=[ 
        lb.Classification(
            class_type=lb.Classification.Type.CHECKLIST,
            instructions="checklist_class",
            scope = lb.Classification.Scope.INDEX, ## Need to defined scope for frame classifications
            options=[ 
                lb.Option(value="first_checklist_answer"),
                lb.Option(value="second_checklist_answer")
            ]
        ),
        lb.Classification(
            class_type=lb.Classification.Type.RADIO,
            instructions="radio_class_global",
            options=[ 
                lb.Option(value="first_radio_answer"),
                lb.Option(value="second_radio_answer")
            ]
        ),
         lb.Classification(
              class_type=lb.Classification.Type.RADIO,
              instructions="radio_question_nested",
              options=[
                  lb.Option("first_radio_question",
                        options=[
                            lb.Classification(
                                class_type=lb.Classification.Type.RADIO,
                                instructions="sub_question_radio",
                                options=[lb.Option("sub_answer")]
                            )
                        ]
                  )
              ] 
        )  
    ]  
)

ontology = client.create_ontology("Ontology Video Annotations", ontology_builder.asdict())

### Step 3: Create a labeling project 
Connect the ontology to the labeling project.

In [40]:
# Project defaults to batch mode with benchmark quality settings if this argument is not provided
# Queue mode will be deprecated once dataset mode is deprecated

project = client.create_project(name="video_project_demo",
                                    media_type=lb.MediaType.Video)

## connect ontology to your project
project.setup_editor(ontology)

######################### DATASET CONSENSUS OPTION ########################
# Note that dataset base projects will be deprecated in the near future.

# To use Datasets/Consensus instead of Batches/Benchmarks use the following query: 
# In this case, 10% of all data rows need to be annotated by three labelers.

# dataset_project = client.create_project(name="dataset-test-project",
#                                 description="a description",
#                                 media_type=MediaType.Text,
#                                 auto_audit_percentage=0.1,
#                                 auto_audit_number_of_labels=3,
#                                 queue_mode=QueueMode.Dataset)

# dataset_project.datasets.connect(dataset)

Default createProject behavior will soon be adjusted to prefer batch projects. Pass in `queue_mode` parameter explicitly to opt-out for the time being.


### Step 4: Send a batch of data rows to the project

In [41]:
# Create a batch to send to your MAL project
batch = project.create_batch(
  "first-batch-video-demo2", # Each batch in a project must have a unique name
  dataset.export_data_rows(), # A paginated collection of data row objects
  5 # priority between 1(Highest) - 5(lowest)
)

print("Batch: ", batch)

Batch:  <Batch {
    "consensus_settings_json": "{\"numberOfLabels\":1,\"coveragePercentage\":0}",
    "created_at": "2023-02-06 16:13:01+00:00",
    "name": "first-batch-video-demo2",
    "size": 1,
    "uid": "2124cd00-a639-11ed-a64b-b30241daddff",
    "updated_at": "2023-02-06 16:13:01+00:00"
}>


### Step 5: Create the annotations payload 
Create the annotations payload using the snippets of code above.

Labelbox supports two formats for the annotations payload: NDJSON and Python Annotation types. However, for video assets, only NDJSON format is supported.

#### NDJSON annotations
Here we create the complete `label_ndjson` payload of annotations. There is one annotation for each *reference to an annotation* that we created [above](https://colab.research.google.com/drive/1S3HBa10jwfFFNB71Uid-crUDdhfEGnXh#scrollTo=kLT9P-WYk4Nr&line=1&uniqifier=1).

In [42]:
## For nested frame base classifications we need to pass a featureSchemaId instead of the name. 

features = project.ontology().normalized

for i in features['tools']:
  print(i)
  if i['name'] == 'bbox_class':
    ## Classification feature schema id
    class_feature_schema_id = i['classifications'][0]['featureSchemaId']
    ## Answer feature schema id (select one of the answers)
    class_options_feature_schema_id = i['classifications'][0]['options'][0]['featureSchemaId']

    ## Update the original annotation with the schema ids
    for frame in frame_bbox_with_checklist_subclass_ndjson['segments']:
      for k in frame['keyframes']:
        k['classifications'][0].update(
            {'schemaId': class_feature_schema_id , 
              'answer': {'schemaId': class_options_feature_schema_id}
              }
            )
        

{'schemaNodeId': 'cldt0hjgu3v3l07351atx6nh5', 'featureSchemaId': 'cldt0hjgu3v3k07352vyz7olj', 'required': False, 'name': 'bbox_video', 'tool': 'rectangle', 'color': '#ff0000', 'archived': 0, 'classifications': []}
{'schemaNodeId': 'cldt0hjgu3v3n07351hg85x4j', 'featureSchemaId': 'cldt0hjgu3v3m07353pkq04j5', 'required': False, 'name': 'point_video', 'tool': 'point', 'color': '#7fff00', 'archived': 0, 'classifications': []}
{'schemaNodeId': 'cldt0hjgu3v3p0735ebm82fg2', 'featureSchemaId': 'cldt0hjgu3v3o0735fwcb8nbw', 'required': False, 'name': 'line_video_frame', 'tool': 'line', 'color': '#00ffff', 'archived': 0, 'classifications': []}
{'schemaNodeId': 'cldt0hjgv3v3z0735fr3f155v', 'featureSchemaId': 'cldt0hjgu3v3q0735azh04wnm', 'required': False, 'name': 'bbox_class', 'tool': 'rectangle', 'color': '#7f00ff', 'archived': 0, 'classifications': [{'schemaNodeId': 'cldt0hjgv3v3y07354odraqgc', 'featureSchemaId': 'cldt0hjgu3v3r07354dxxbq6p', 'archived': 0, 'required': False, 'instructions': 'bbox

In [43]:
label_ndjson = []

for annotations in [point_annotation_ndjson,
                    bbox_annotation_ndjson,
                    polyline_frame_annotation_ndjson, 
                    frame_checklist_classification_ndjson, 
                    global_radio_classification_ndjson,
                    nested_classification,
                    frame_bbox_with_checklist_subclass_ndjson
                    ]:      
  annotations.update({
      'uuid' : str(uuid.uuid4()),
      'dataRow': {
          'id':  next(dataset.export_data_rows()).uid
      }
  })
  label_ndjson.append(annotations)


In [44]:
label_ndjson

[{'name': 'point_video',
  'segments': [{'keyframes': [{'frame': 17,
      'point': {'x': 660.134, 'y': 407.926}}]}],
  'uuid': '8ea629d0-a911-4666-b5f0-358749c9a549',
  'dataRow': {'id': 'cldt0g5pd06gd07z22m2i6k66'}},
 {'name': 'bbox_video',
  'segments': [{'keyframes': [{'frame': 13,
      'bbox': {'top': 146.0, 'left': 98.0, 'height': 382.0, 'width': 341.0}},
     {'frame': 14,
      'bbox': {'top': 146.0, 'left': 98.0, 'height': 382.0, 'width': 341.0}},
     {'frame': 15,
      'bbox': {'top': 146.0,
       'left': 98.0,
       'height': 382.0,
       'width': 341.0}}]}],
  'uuid': '3bcc159e-64ba-45fd-bfb7-687a01535726',
  'dataRow': {'id': 'cldt0g5pd06gd07z22m2i6k66'}},
 {'name': 'line_video_frame',
  'segments': [{'keyframes': [{'frame': 5,
      'line': [{'x': 680, 'y': 100},
       {'x': 100, 'y': 190},
       {'x': 190, 'y': 220}]},
     {'frame': 12,
      'line': [{'x': 680, 'y': 280},
       {'x': 300, 'y': 380},
       {'x': 400, 'y': 460}]},
     {'frame': 20,
      'line

### Step 6: Upload annotations to a project as pre-labels or completed labels
For the purpose of this tutorial only run one of the label imports at once, otherwise the previous import might get overwritten.

#### Model-Assisted Labeling (MAL)

In [45]:
# Upload MAL label for this data row in project
upload_job_mal = lb.MALPredictionImport.create_from_objects(
    client = client, 
    project_id = project.uid, 
    name="mal_import_job-" + str(uuid.uuid4()), 
    predictions=label_ndjson)

upload_job_mal.wait_until_done();
print("Errors:", upload_job_mal.errors)
print("   ")

Errors: []
   


#### Label Import

In [46]:
upload_job_label_import = lb.LabelImport.create_from_objects(
    client = client,
    project_id = project.uid, 
    name = "label_import_job-" + str(uuid.uuid4()),
    labels = label_ndjson
)

upload_job_label_import.wait_until_done();
print("Errors:", upload_job_label_import.errors)
print("   ")

Errors: []
   


### Optional deletions for cleanup

In [19]:
# project.delete()
# dataset.delete()